In [1]:
# General libraries imports
import pandas as pd
import numpy as np
import os
import time

# import image manipulation
from PIL import Image

# Import PyTorch
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

In [16]:
TRAIN_IMG_PATH = "./data/train/"
TEST_IMG_PATH = "./data/test/"
LABELS_CSV_PATH = "./data/train.csv"
SAMPLE_SUB_PATH = "./data/sample_submission.csv"

class CactusDataset(Dataset):
    """Cactus identification dataset."""

    def __init__(self, img_dir, dataframe, transform=None):
        """
        Args:
            img_dir (string): Directory with all the images.        
            dataframe (pandas.core.frame.DataFrame): Pandas dataframe obtained
                by read_csv().
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_frame = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_frame.id[idx]) 
        image = Image.open(img_name)
        label = self.labels_frame.has_cactus[idx]

        if self.transform:
            image = self.transform(image)

        return [image, label] 
    
    
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

In [17]:
dframe = pd.read_csv(LABELS_CSV_PATH)
cut = int(len(dframe)*0.95)
train, test = np.split(dframe, [cut], axis=0)
test = test.reset_index(drop=True)

train_ds = CactusDataset(TRAIN_IMG_PATH, train, train_transforms)
test_ds = CactusDataset(TRAIN_IMG_PATH, test, test_transforms)
datasets = {"train": train_ds, "val": test_ds}

In [18]:
trainloader = DataLoader(train_ds, batch_size=32,
                        shuffle=True, num_workers=0)

testloader = DataLoader(test_ds, batch_size=4,
                        shuffle=True, num_workers=0)

In [23]:
epochs = 1
batch_size = 128
learning_rate = 0.003
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [24]:
class ModelCNN(nn.Module):

    #Our batch shape for input x is (32, 32, 3)

    def __init__(self):
        '''
        Init method
        '''
        super(ModelCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=0, padding=0)

        self.fc1 = nn.Linear(128 * 25 * 25, 512)
        
        self.dropout1 = nn.Dropout(0.3)

    def forward(self, x):
        '''
        Forward pass of the model.
        INPUT:
            x - input data
        '''
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        
        x = self.pool(x)
        
        x = F.relu(self.conv5(x))
        
        x = self.pool(x)

        x = x.view(-1, 128 * 25 * 25)

        x = F.relu(self.fc1(x))

        x = F.log_softmax(self.dropout1(x), dim = 1)
        
        return(x)

In [25]:
model = ModelCNN()

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
model.to(device)

steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 4.795.. Test loss: 1.051.. Test accuracy: 0.749
Epoch 1/1.. Train loss: 2.514.. Test loss: 1.611.. Test accuracy: 0.748
Epoch 1/1.. Train loss: 2.751.. Test loss: 1.492.. Test accuracy: 0.749
Epoch 1/1.. Train loss: 2.897.. Test loss: 1.237.. Test accuracy: 0.749
Epoch 1/1.. Train loss: 2.701.. Test loss: 1.179.. Test accuracy: 0.749
Epoch 1/1.. Train loss: 2.059.. Test loss: 1.076.. Test accuracy: 0.748
Epoch 1/1.. Train loss: 2.277.. Test loss: 1.437.. Test accuracy: 0.811
Epoch 1/1.. Train loss: 2.373.. Test loss: 0.807.. Test accuracy: 0.870
Epoch 1/1.. Train loss: 2.237.. Test loss: 1.479.. Test accuracy: 0.816
Epoch 1/1.. Train loss: 2.315.. Test loss: 0.956.. Test accuracy: 0.885
Epoch 1/1.. Train loss: 2.062.. Test loss: 0.999.. Test accuracy: 0.873
Epoch 1/1.. Train loss: 2.045.. Test loss: 1.315.. Test accuracy: 0.892
Epoch 1/1.. Train loss: 2.381.. Test loss: 0.993.. Test accuracy: 0.888
Epoch 1/1.. Train loss: 2.310.. Test loss: 1.053.. Test accuracy

In [27]:
submission_df = pd.read_csv(SAMPLE_SUB_PATH)
output_df = pd.DataFrame(index=submission_df.index, columns=submission_df.keys() )
output_df['id'] = submission_df['id']
submission_df['target'] =  [0] * len(submission_df)

tdata_transform = transforms.Compose([transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

submission_ds = CactusDataset(TEST_IMG_PATH, submission_df, tdata_transform)

sub_loader = DataLoader(submission_ds, batch_size=1,
                        shuffle=False, num_workers=0)


def test_sumission(model):
    since = time.time()
    sub_outputs = []
    model.train(False)  # Set model to evaluate mode
    # Iterate over data.
    prediction = []
    for data in sub_loader:
        # get the inputs
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        prediction.append(int(pred))
      
    time_elapsed = time.time() - since
    print('Run complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return prediction

In [29]:
sub = pd.read_csv('./data/sample_submission.csv')
sub['has_cactus'] = test_sumission(model)
sub.to_csv('submission_conv.csv', index= False)

sub.head()

Run complete in 2m 49s


,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,1
1,0017242f54ececa4512b4d7937d1e21e.jpg,1
2,001ee6d8564003107853118ab87df407.jpg,0
3,002e175c3c1e060769475f52182583d0.jpg,0
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,1
